# Mixed Integer Program (MIP)

Reference: Reference: https://developers.google.com/optimization/lp/lp_example
\begin{align*}
\text{max} \quad & x + 10y \\
\text{s.t.} \quad & x + 7y \leq 17.5 \\
                  & 0 \leq x \leq 3.5 \\
                  & 0 \leq y \\
                  & x, y \text{ integers}
\end{align*}

<div style="text-align: center;">
  <img src="https://developers.google.com/static/optimization/images/mip/feasible_region_sol.png">
</div>

## SciPy

In [15]:
from scipy.optimize import linprog
import numpy as np

# Objective function:
c = [-1, -10]

# Constraints:
A = [[1, 7]]
b = [17.5]
x_bounds = (0, 3.5)
y_bounds = (0, None)

# Solve the relaxed problem (continuous)
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

# If the relaxed problem has a solution, we need to check integer solutions:
if result.success:
    # Generate all possible integer pairs (x, y) within bounds
    x_values = np.arange(0, 4)  # x can be 0, 1, 2, 3 (since 3.5 is non-integer)
    y_values = np.arange(0, int(b[0]/7) + 1)
    
    best_value = -np.inf
    best_solution = None
    
    for x in x_values:
        for y in y_values:
            if x + 7 * y <= 17.5:
                value = x + 10 * y
                if value > best_value:
                    best_value = value
                    best_solution = (x, y)
    
    if best_solution:
        print(f"Optimal value: {best_value}")
        print(f"Optimal x: {best_solution[0]}")
        print(f"Optimal y: {best_solution[1]}")
    else:
        print("No feasible integer solution found.")
else:
    print("The problem does not have an optimal solution.")


Optimal value: 23
Optimal x: 3
Optimal y: 2


## CVXPY

In [16]:
import cvxpy as cp

# Variable Definition:
x = cp.Variable(integer=True)
y = cp.Variable(integer=True)

# Objective Function:
objective = cp.Maximize(x + 10 * y)

# Constraints:
constraints = [
    x + 7 * y <= 17.5,
    0 <= x,
    x <= 3.5,
    0 <= y
]

# Solve:
problem = cp.Problem(objective, constraints)
problem.solve()

if problem.status == cp.OPTIMAL:
    print(f"Optimal value: {problem.value}")
    print(f"Optimal x: {x.value}")
    print(f"Optimal y: {y.value}")
else:
    print("The problem does not have an optimal solution.")


Optimal value: 23.0
Optimal x: 3.0
Optimal y: 2.0


## PuLP

In [17]:
import pulp

# Create a problem
problem = pulp.LpProblem("Maximize_xy", pulp.LpMaximize)

# Variable Definition:
x = pulp.LpVariable('x', lowBound=0, upBound=3.5, cat='Integer')
y = pulp.LpVariable('y', lowBound=0, cat='Integer')

# Objective Function:
problem += x + 10 * y

# Constraints:
problem += x + 7 * y <= 17.5

# Solve:
status = problem.solve()

if pulp.LpStatus[status] == 'Optimal':
    print(f"Optimal value: {pulp.value(problem.objective)}")
    print(f"Optimal x: {pulp.value(x)}")
    print(f"Optimal y: {pulp.value(y)}")
else:
    print("The problem does not have an optimal solution.")


Optimal value: 23.0
Optimal x: 3.0
Optimal y: 2.0


## Pyomo

In [18]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, maximize, NonNegativeIntegers

# Create a model
model = ConcreteModel()

# Variable Definition:
model.x = Var(within=NonNegativeIntegers, bounds=(0, 3))
model.y = Var(within=NonNegativeIntegers)

# Objective Function:
model.objective = Objective(expr=model.x + 10 * model.y, sense=maximize)

# Constraints:
model.constraint1 = Constraint(expr=model.x + 7 * model.y <= 17.5)

# Solve:
solver = SolverFactory('glpk')
status = solver.solve(model)

if (status.solver.termination_condition == 'optimal') or (status.solver.termination_condition == 'feasible'):
    print(f"Optimal value: {model.objective()}")
    print(f"Optimal x: {model.x()}")
    print(f"Optimal y: {model.y()}")
else:
    print("The problem does not have an optimal solution.")


Optimal value: 23.0
Optimal x: 3.0
Optimal y: 2.0


## Google OR-Tools

In [19]:
from ortools.linear_solver import pywraplp

# Create a solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variable Definition:
x = solver.IntVar(0, 3.5, 'x')
y = solver.IntVar(0, solver.infinity(), 'y')

# Objective Function:
solver.Maximize(x + 10 * y)

# Constraints:
solver.Add(x + 7 * y <= 17.5)

# Solve:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print(f"Optimal value: {solver.Objective().Value()}")
    print(f"Optimal x: {x.solution_value()}")
    print(f"Optimal y: {y.solution_value()}")
else:
    print("The problem does not have an optimal solution.")


Optimal value: 23.0
Optimal x: 3.0
Optimal y: 2.0


## Gekko

In [20]:
from gekko import GEKKO

# Create a model
m = GEKKO()

# Variable Definition:
x = m.Var(value=0, lb=0, ub=3.5, integer=True)
y = m.Var(value=0, lb=0, integer=True)

# Objective Function:
m.Maximize(x + 10 * y)

# Constraints:
m.Equation(x + 7 * y <= 17.5)

# Solve:
m.options.SOLVER = 1
m.solve(disp=False)

print(f"Optimal value: {m.options.objfcnval * -1}")
print(f"Optimal x: {x.value[0]}")
print(f"Optimal y: {y.value[0]}")


Optimal value: 23.0
Optimal x: 3.0
Optimal y: 2.0
